In [2]:
%matplotlib notebook

import matplotlib.pylab as plt
import numpy as np
import seaborn as sns; sns.set()

import keras
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam
import salty
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
devmodel = salty.aggregate_data(['thermal_conductivity'], T=[100, 700], P=[99, 102])
devmodel.Data['smiles_string'] = devmodel.Data['smiles-cation'] + "." + devmodel.Data['smiles-anion']
values = devmodel.Data['smiles_string']

C:\Users\james\Anaconda3\lib\site-packages\pandas\core\generic.py:4388: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
C:\Users\james\Anaconda3\lib\site-packages\pandas\core\generic.py:4389: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)


In [4]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

In [9]:
print(values[0])

CCCCCCCCn1cc[n+](c1)C.[B-](F)(F)(F)F


In [5]:
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


C:\Users\james\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [17]:
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print("input:", values[0])
print("inverted:", inverted)

input: CCCCCCCCn1cc[n+](c1)C.[B-](F)(F)(F)F
inverted: ['CCCCCCCCn1cc[n+](c1)C.[B-](F)(F)(F)F']


In [19]:
x_train = onehot_encoded

In [20]:
X_train, X_test = train_test_split(x_train, test_size=0.33, random_state=42)

In [21]:
X_train.shape

(317, 61)

In [22]:
m = Sequential()
m.add(Dense(20,  activation='elu', input_shape=(61,)))
m.add(Dense(2,    activation='linear', name="bottleneck"))
m.add(Dense(20,  activation='elu'))
m.add(Dense(61,  activation='sigmoid'))
m.compile(loss='mean_squared_error', optimizer = Adam())
history = m.fit(x_train, x_train, batch_size=20, epochs=5, verbose=1, 
                validation_data=(X_test, X_test))

encoder = Model(m.input, m.get_layer('bottleneck').output)
Zenc = encoder.predict(x_train)  # bottleneck representation
Renc = m.predict(x_train)        # reconstruction

Train on 474 samples, validate on 157 samples
Epoch 1/5
474/474 [==============================] - 0s 1ms/step - loss: 0.2436 - val_loss: 0.2346
Epoch 2/5
474/474 [==============================] - 0s 105us/step - loss: 0.2176 - val_loss: 0.1942
Epoch 3/5
474/474 [==============================] - 0s 102us/step - loss: 0.1602 - val_loss: 0.1205
Epoch 4/5
474/474 [==============================] - 0s 102us/step - loss: 0.0850 - val_loss: 0.0541
Epoch 5/5
474/474 [==============================] - 0s 93us/step - loss: 0.0384 - val_loss: 0.0275
